# Shopping cart analysis

Problem Description: [Link in Quera](https://quera.org/problemset/125313/)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./data/supermarket.zip',
                 # dtype={'Product':'category'},
                 # parse_dates=['Date']
                )

In [4]:
df

,Customer Id,Date,Product
0,JKML5ZWXP0XQ,2020-07-20,tropical fruit
1,YOGWQIUPDAJ7,2020-04-30,whole milk
2,Q3NL22281AF6,2020-09-18,pip fruit
3,C59GRTLMZQAU,2020-12-11,other vegetables
4,WUSF8LTEQZ5L,2020-01-02,whole milk
...,...,...,...
32705,PM1BQU9H9S2Q,2019-06-05,oil
32706,6AWIAP3ZV551,2019-08-10,sliced cheese
32707,LRWVFLSC8X5B,2019-02-23,candy
32708,8KYRMELYKNIH,2019-04-16,cake bar


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32710 entries, 0 to 32709
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer Id  32710 non-null  object
 1   Date         32710 non-null  object
 2   Product      32710 non-null  object
dtypes: object(3)
memory usage: 766.8+ KB


## <div direction=rtl, align=right> تعداد محصولات یکتا در کل دادگان، چندتاست؟ </div>

In [5]:
q1 = df.Product.nunique()
q1

137

## <div direction=rtl, align=right> میزان فروش به ازای هر روز به صورت میانگین چند است (تا دو رقم بعد از اعشار)؟ </div>

In [13]:
res = df.Product.size / df.Date.nunique()
q2 = round(res, 2)
q2

44.93

In [6]:
q2 = (df[['Date', 'Product']]
      .groupby('Date')
      .count()
      .mean()
      .round(2)
      .item()
     )
q2

44.93

## <div direction=rtl, align=right> چهار محصولی که کمتر از بقیه در سبد مشتریان قرار گرفته‌اند، کدامند (ترتیب مهم نیست)؟ </div>

In [7]:
sales_of_each_product = df.Product.value_counts(ascending=True)

In [8]:
q3 = ','.join(
    map(
        str,
        sales_of_each_product[:4].index
    )
)
q3

'preservation products,kitchen utensil,baby cosmetics,bags'

## <div direction=rtl, align=right> پنج مشتری‌ای که در سال ۲۰۲۰، بیشترین تعداد "سبد" را داشته‌اند، کدامند (ترتیب مهم نیست)؟ </div>

In [9]:
df.sort_values('Date', inplace=True)

In [10]:
df.loc[df.Date>'2020']

,Customer Id,Date,Product
14554,BNREUVSNK4KI,2020-01-01,brown bread
2944,JCDI7PFUKHTO,2020-01-01,sausage
13322,2T4TPX1TFIJT,2020-01-01,tropical fruit
15759,LAOPYEIMY8JU,2020-01-01,turkey
18764,O5HGE614NF7Q,2020-01-01,pickled vegetables
...,...,...,...
14588,2O2YLJ5FUAJT,2020-12-29,bottled beer
17620,5USJ20248CHY,2020-12-29,herbs
29635,0LRNCT5AF6LF,2020-12-29,rolls/buns
6344,NV334BJ2XM2Z,2020-12-29,tropical fruit


In [16]:
Customer2020 = (
    df
    .loc[df.Date>'2020', ['Customer Id', 'Date']]
    .groupby(['Customer Id'])
    .nunique()
)

In [17]:
Customer2020

,Date
Customer Id,
0011457U6R3Y,1
00D4E3I1MS2R,3
011K5G14VFYQ,1
018OA73Z6TR6,3
01HFRBUAX0R7,1
...,...
ZYK1C5SUQ37I,1
ZYLKVTPXLLUH,4
ZYPGG9P8NXLT,1


In [14]:
df2 = df.loc[df.Date.str.contains('2020')]

In [15]:
df2.value_counts('Customer Id')

Customer Id
0ZRF32AJ06BC    26
KACGV5OJXMJW    23
0P1TPIH74E79    21
3HRBE45N5NB6    20
ED8WY0DE27XG    20
                ..
9F72DLDOMRJQ     1
3RYPXAQGHMWN     1
RHH9PMYZLJ0K     1
X678EDW1N59W     1
7ERD4BG8GASE     1
Length: 3298, dtype: int64

In [19]:
Customer2020.nlargest(5, 'Date')

,Date
Customer Id,
0ZRF32AJ06BC,7
E7FD9BU4OK0F,7
M2DT2Z00NQCH,7
M722MPITBY07,7
QBUS9NZY39BE,7


In [12]:
q4 = ','.join(
    Customer2020
    .sort_values(by='Date', ascending=False)
    .head(5)
    .index
)
q4

'M722MPITBY07,E7FD9BU4OK0F,0ZRF32AJ06BC,M2DT2Z00NQCH,QBUS9NZY39BE'

## <div direction=rtl, align=right> کدام روز هفته، بیشترین تعداد محصول فروش رفته‌است؟ </div>

In [20]:
df.Date.head()

0    2020-07-20
1    2020-04-30
2    2020-09-18
3    2020-12-11
4    2020-01-02
Name: Date, dtype: object

In [13]:
%%timeit
df_day = df.Date.apply(lambda x: pd.to_datetime(x).strftime("%A"))

3.82 s ± 7.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
df_day = (
    pd.to_datetime(df.Date)
    .dt.day_name()
)

16.5 ms ± 206 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
df_day = (
    pd.to_datetime(df.Date)
    .dt.day_name()
)
df_day

0           Monday
1         Thursday
2           Friday
3           Friday
4         Thursday
           ...    
32705    Wednesday
32706     Saturday
32707     Saturday
32708      Tuesday
32709      Tuesday
Name: Date, Length: 32710, dtype: object

In [16]:
df_day.value_counts()

Wednesday    4846
Tuesday      4802
Saturday     4774
Monday       4655
Thursday     4614
Friday       4539
Sunday       4480
Name: Date, dtype: int64

In [17]:
q5 = df_day.value_counts().index[0]
q5

'Wednesday'

## <div style="direction:rtl;align:right"> پنج محصولی که بیشترین support را دارند به ترتیب نزولی کدامند؟ </div>

In [25]:
basket = df.groupby(['Customer Id', 'Date'])
n_basket = basket.ngroups
n_basket

14753

In [26]:
df_pd = df[['Product', 'Date']].value_counts()

In [27]:
df_pd

Product           Date      
whole milk        2020-01-04    13
                  2020-08-27    11
                  2020-08-18    10
                  2020-11-27    10
                  2020-11-20     9
                                ..
organic products  2019-10-22     1
                  2019-12-04     1
                  2020-01-03     1
                  2020-03-06     1
curd              2019-05-03     1
Length: 21335, dtype: int64

In [28]:
n_product_in_baskets = (
    df_pd
    .groupby(level=0)
    .sum()
    .sort_values(ascending=False)
)
n_product_in_baskets

Product
whole milk               2363
other vegetables         1827
rolls/buns               1646
soda                     1453
yogurt                   1285
                         ... 
make up remover             5
bags                        4
baby cosmetics              3
preservation products       1
kitchen utensil             1
Length: 137, dtype: int64

In [29]:
product_support = n_product_in_baskets / n_basket
product_support

Product
whole milk               0.160171
other vegetables         0.123839
rolls/buns               0.111571
soda                     0.098488
yogurt                   0.087101
                           ...   
make up remover          0.000339
bags                     0.000271
baby cosmetics           0.000203
preservation products    0.000068
kitchen utensil          0.000068
Length: 137, dtype: float64

In [29]:
q6 = ','.join(product_support[:5].index)
q6

'whole milk,other vegetables,rolls/buns,soda,yogurt'

In [31]:
(product_support >= 0.01).sum()

53

## <div style="direction:rtl;align:right"> از بین مواردی که support آن‌ها، حداقل ۰.۰۱ می‌باشد. دو الگویی که بیشترین confidence را دارند، به ترتیب نزولی کدامند؟ </div>

### Naive method:

In [68]:
high_product_support = product_support[product_support > 0.01]
high_product_support = list(map(lambda x:set((x,)),high_product_support.index))

In [54]:
baskets_list =[frozenset(b.Product) for a, b in basket]

In [69]:
high_sups = []
for name1 in high_product_support:
    for name2, sup2 in high_sups:
        break
    high_sups.append((name1, sup1))
    break
high_sups

[({'whole milk'}, 0.16017081271605776)]

In [63]:
high_product_support

{'UHT-milk',
 'beef',
 'berries',
 'beverages',
 'bottled beer',
 'bottled water',
 'brown bread',
 'butter',
 'butter milk',
 'candy',
 'chewing gum',
 'chicken',
 'chocolate',
 'citrus fruit',
 'coffee',
 'curd',
 'dessert',
 'domestic eggs',
 'frozen meals',
 'frozen vegetables',
 'fruit/vegetable juice',
 'grapes',
 'hamburger meat',
 'hard cheese',
 'herbs',
 'hygiene articles',
 'ice cream',
 'long life bakery product',
 'margarine',
 'meat',
 'napkins',
 'oil',
 'onions',
 'other vegetables',
 'pastry',
 'pip fruit',
 'processed cheese',
 'rolls/buns',
 'root vegetables',
 'salty snack',
 'sausage',
 'sliced cheese',
 'soda',
 'soft cheese',
 'specialty bar',
 'specialty chocolate',
 'sugar',
 'tropical fruit',
 'waffles',
 'whipped/sour cream',
 'white bread',
 'whole milk',
 'yogurt'}

In [57]:
high_product_support[name1]

0.16017081271605776

### mlxtend

## Output:

In [ ]:
print(*(eval(f'q{i}') for i in range(1,8)), sep='\n')

In [ ]:
with open('output.txt', 'w') as file:
    print(*(eval(f'q{i}') for i in range(1,7)), sep='\n', file=file)